<a href="https://colab.research.google.com/github/soham7707/DeepSectorAI/blob/main/GRU_LSTM_BiLstm_HybridwithGARCH_Beta_Bearish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.3/978.3 kB 12.4 MB/s eta 0:00:00


In [2]:
pip install yfinance pandas numpy matplotlib

In [5]:
sectors = {
    'IT': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS'],
    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
    'Pharma': ['SUNPHARMA.NS', 'CIPLA.NS', 'DRREDDY.NS'],
    'Auto': ['TATAMOTORS.NS', 'M&M.NS', 'MARUTI.NS'],
    'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'DABUR.NS']
}

In [7]:
import yfinance as yf
import pandas as pd
from datetime import datetime

#Bearish Period

start_date = "2022-01-01"
end_date = "2022-06-30"

def fetch_sector_data(sectors):
    all_data = {}
    for sector, stocks in sectors.items():
        print(f"Fetching data for {sector} sector...")
        data = yf.download(stocks, start=start_date, end=end_date)['Close']
        all_data[sector] = data
    return all_data

sector_data = fetch_sector_data(sectors)


Fetching data for IT sector...


/tmp/ipython-input-4068912510.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']
[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-4068912510.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Banking sector...


[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-4068912510.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']
[                       0%                       ]

Fetching data for Pharma sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-4068912510.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Auto sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-4068912510.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for FMCG sector...


[*********************100%***********************]  3 of 3 completed


In [8]:

for sector in sector_data:
    df = sector_data[sector]
    df = df.ffill().bfill()
    sector_data[sector] = df


In [9]:

import numpy as np

returns_data = {}

for sector, df in sector_data.items():
    returns = np.log(df / df.shift(1)).dropna()
    returns_data[sector] = returns


In [10]:
for sector, df in returns_data.items():
    df.to_csv(f"{sector}_returns.csv")

In [11]:
sector_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2022-01-03,693.454590,1449.202515,741.773926,1817.761841
2022-01-04,706.201355,1457.689941,749.679626,1845.808472
2022-01-05,723.877563,1492.307007,764.423889,1915.103638
2022-01-06,727.263428,1468.370728,761.513794,1884.964722
2022-01-07,727.562134,1478.670044,769.468018,1897.269287
...,...,...,...,...
2022-06-23,626.733093,1288.059692,678.189209,1677.926392
2022-06-24,631.363892,1306.052002,692.060486,1692.173950
2022-06-27,634.450867,1308.415527,695.843506,1685.349121


In [12]:
returns_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2022-01-04,0.018215,0.005840,0.010601,0.015311
2022-01-05,0.024722,0.023470,0.019477,0.036854
2022-01-06,0.004666,-0.016170,-0.003814,-0.015863
2022-01-07,0.000411,0.006990,0.010391,0.006507
2022-01-10,0.016561,0.005531,0.021821,0.022689
...,...,...,...,...
2022-06-23,0.004060,0.003677,0.018114,0.008587
2022-06-24,0.007362,0.013872,0.020247,0.008455
2022-06-27,0.004877,0.001808,0.005451,-0.004041


In [13]:
from arch import arch_model
import pandas as pd
import numpy as np

In [14]:
def estimate_garch_volatility(returns_series):
    # Fit GARCH(1,1) model to the returns series
    model = arch_model(returns_series, vol='GARCH', p=1, q=1)
    res = model.fit(disp='off')
    # Return the estimated conditional volatility (σ_t)
    return res.conditional_volatility

In [15]:
garch_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Estimating GARCH volatility for {sector} sector...")
    sector_vols = pd.DataFrame(index=returns_df.index)

    for stock in returns_df.columns:
        try:
            vol = estimate_garch_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error processing {stock}: {e}")

    garch_volatility_data[sector] = sector_vols

Estimating GARCH volatility for IT sector...
Estimating GARCH volatility for Banking sector...
Estimating GARCH volatility for Pharma sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003374. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003659. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

Estimating GARCH volatility for Auto sector...
Estimating GARCH volatility for FMCG sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004004. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004512. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:309: DataScaleWarn

In [16]:
print(garch_volatility_data['FMCG'].tail())

            DABUR.NS  HINDUNILVR.NS    ITC.NS
Date                                         
2022-06-23  0.017312       0.018421  0.014784
2022-06-24  0.016515       0.018424  0.014784
2022-06-27  0.015483       0.018426  0.014784
2022-06-28  0.015254       0.018429  0.014784
2022-06-29  0.014115       0.018431  0.014784


In [17]:
pip install numpy pandas scikit-learn tensorflow

In [18]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_lstm_data(series, lookback=20):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)
    return X, y, scaler

In [19]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [20]:
def predict_lstm_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)
    model = build_lstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    # Estimate volatility as rolling std of predicted returns
    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility

In [21]:
lstm_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting LSTM volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_lstm_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    lstm_volatility_data[sector] = sector_vols

Predicting LSTM volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


In [22]:
print(lstm_volatility_data['IT'].tail())

            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2022-06-23    0.002434  0.002820  0.002281  0.003461
2022-06-24    0.002222  0.002663  0.002196  0.003538
2022-06-27    0.002168  0.002511  0.002156  0.003592
2022-06-28    0.002169  0.002459  0.002177  0.003571
2022-06-29    0.002187  0.002468  0.002209  0.003482


In [23]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import LSTM, Dense, Bidirectional


def build_bilstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=50, return_sequences=False), input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [24]:
def predict_bilstm_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)
    model = build_bilstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility

In [25]:
bilstm_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting BiLSTM volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_bilstm_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    bilstm_volatility_data[sector] = sector_vols

Predicting BiLSTM volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step


In [26]:
print(bilstm_volatility_data['IT'].tail())

            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2022-06-23    0.001742  0.001399  0.001495  0.001873
2022-06-24    0.001752  0.001230  0.001476  0.001684
2022-06-27    0.001656  0.001029  0.001413  0.001548
2022-06-28    0.001649  0.000991  0.001388  0.001576
2022-06-29    0.001637  0.001001  0.001382  0.001584


In [27]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import GRU, Dense



def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(units=50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model



In [28]:
def predict_gru_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)  # same prepare function works
    model = build_gru_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility

In [29]:
gru_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting GRU volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_gru_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    gru_volatility_data[sector] = sector_vols

Predicting GRU volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


In [30]:
print(gru_volatility_data['IT'].tail())


            HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2022-06-23    0.003390  0.004082  0.004634  0.004942
2022-06-24    0.003118  0.003979  0.004787  0.004931
2022-06-27    0.003013  0.003951  0.004946  0.004913
2022-06-28    0.003011  0.003960  0.004970  0.004768
2022-06-29    0.002858  0.003789  0.004908  0.004558


In [31]:
gru_volatility_data['IT'].dropna

<bound method DataFrame.dropna of             HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS
Date                                                
2022-02-02         NaN       NaN       NaN       NaN
2022-02-03         NaN       NaN       NaN       NaN
2022-02-04         NaN       NaN       NaN       NaN
2022-02-07         NaN       NaN       NaN       NaN
2022-02-08         NaN       NaN       NaN       NaN
...                ...       ...       ...       ...
2022-06-23    0.003390  0.004082  0.004634  0.004942
2022-06-24    0.003118  0.003979  0.004787  0.004931
2022-06-27    0.003013  0.003951  0.004946  0.004913
2022-06-28    0.003011  0.003960  0.004970  0.004768
2022-06-29    0.002858  0.003789  0.004908  0.004558

[101 rows x 4 columns]>

In [32]:
gru_volatility_data['IT'].tail()


,HCLTECH.NS,INFY.NS,TCS.NS,WIPRO.NS
Date,,,,
2022-06-23,0.003390,0.004082,0.004634,0.004942
2022-06-24,0.003118,0.003979,0.004787,0.004931
2022-06-27,0.003013,0.003951,0.004946,0.004913
2022-06-28,0.003011,0.003960,0.004970,0.004768
2022-06-29,0.002858,0.003789,0.004908,0.004558


In [33]:
import yfinance as yf
import numpy as np
import pandas as pd

# Download NIFTY-50 index (symbol: ^NSEI)
nifty = yf.download("^NSEI", start="2015-01-01")['Close']
nifty_returns = np.log(nifty / nifty.shift(1)).dropna()
nifty_returns_df = pd.DataFrame(nifty_returns)
nifty_returns_df.columns = ['Market']

/tmp/ipython-input-2614447981.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nifty = yf.download("^NSEI", start="2015-01-01")['Close']
[*********************100%***********************]  1 of 1 completed


In [34]:
import statsmodels.api as sm

def compute_beta(stock_returns, market_returns_df):
    aligned = pd.concat([stock_returns, market_returns_df], axis=1).dropna()
    aligned.columns = ['Stock', 'Market']

    X = sm.add_constant(aligned['Market'])
    y = aligned['Stock']
    model = sm.OLS(y, X).fit()
    return model.params['Market']  # This is the beta




beta_values = {}

for sector, returns_df in returns_data.items():
    print(f"Computing Beta for {sector} sector...")
    sector_betas = {}

    for stock in returns_df.columns:
        try:
            beta = compute_beta(returns_df[stock], nifty_returns_df)
            sector_betas[stock] = beta
        except Exception as e:
            print(f"Error with {stock}: {e}")

    beta_values[sector] = sector_betas



beta_df = pd.DataFrame(beta_values).T  # Transpose to have stocks as columns
beta_df = beta_df.fillna(0)
print(beta_df)


Computing Beta for IT sector...
Computing Beta for Banking sector...
Computing Beta for Pharma sector...
Computing Beta for Auto sector...
Computing Beta for FMCG sector...
         HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS  AXISBANK.NS  HDFCBANK.NS  \
IT         0.935741  1.029993  0.881973  1.100392     0.000000     0.000000   
Banking    0.000000  0.000000  0.000000  0.000000     1.025736     1.076282   
Pharma     0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
Auto       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
FMCG       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   

         ICICIBANK.NS  KOTAKBANK.NS  CIPLA.NS  DRREDDY.NS  SUNPHARMA.NS  \
IT           0.000000      0.000000  0.000000    0.000000      0.000000   
Banking      1.004769      1.003439  0.000000    0.000000      0.000000   
Pharma       0.000000      0.000000  0.378681    0.732036      0.635612   
Auto         0.000000      0.000000  0.000000    0.0

In [35]:
sharpe_ratios = {}

for sector, df in returns_data.items():
    sector_sharpes = {}
    for stock in df.columns:
        r = df[stock].dropna()
        mean_return = r.mean()
        std_return = r.std()
        sharpe = mean_return / std_return if std_return != 0 else 0
        sector_sharpes[stock] = sharpe
    sharpe_ratios[sector] = sector_sharpes



features = []

for sector in returns_data:
    for stock in returns_data[sector].columns:
        try:
            beta = beta_values[sector][stock]
            garch_mean = garch_volatility_data[sector][stock].mean()
            lstm_mean = lstm_volatility_data[sector][stock].mean()
            bilstm_mean = bilstm_volatility_data[sector][stock].mean()
            gru_mean = gru_volatility_data[sector][stock].mean()
            sharpe = sharpe_ratios[sector][stock]

            features.append([sector, stock, beta, garch_mean, lstm_mean, bilstm_mean, gru_mean, sharpe])
        except Exception as e:
            print(f"Skipping {stock} due to missing data: {e}")

features_df = pd.DataFrame(
    features,
    columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'LSTM_Vol', 'BiLSTM_Vol', 'GRU_Vol', 'Sharpe']
)


In [36]:
features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol','LSTM_Vol', 'BiLSTM_Vol', 'GRU_Vol', 'Sharpe'])

In [37]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)

Learned Weights: [ -0.21262152   3.76720293  52.8923556  -13.14196368]
Intercept: -0.051879274070611416


In [38]:
X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = features_df['Sharpe']

# Drop rows with NaNs
df_clean = pd.concat([X, y], axis=1).dropna()

X = df_clean[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = df_clean['Sharpe']

model = LinearRegression().fit(X, y)
print("Learned Weights:", model.coef_)
print("Intercept:", model.intercept_)

Learned Weights: [ -0.21262152   3.76720293  52.8923556  -13.14196368]
Intercept: -0.051879274070611416


In [39]:
features_df['Risk_Score'] = X @ weights

In [40]:
for sector in features_df['Sector'].unique():
    print(f"\nTop 3 safest stocks in {sector}:")
    print(features_df[features_df['Sector'] == sector].sort_values('Risk_Score').head(3)[['Stock', 'Risk_Score']])


Top 3 safest stocks in IT:
        Stock  Risk_Score
3    WIPRO.NS   -0.039840
0  HCLTECH.NS   -0.017117
1     INFY.NS   -0.013613

Top 3 safest stocks in Banking:
          Stock  Risk_Score
7  KOTAKBANK.NS   -0.034547
6  ICICIBANK.NS   -0.019516
4   AXISBANK.NS    0.034154

Top 3 safest stocks in Pharma:
           Stock  Risk_Score
10  SUNPHARMA.NS   -0.002189
9     DRREDDY.NS    0.015006
8       CIPLA.NS    0.095128

Top 3 safest stocks in Auto:
            Stock  Risk_Score
13  TATAMOTORS.NS   -0.068927
12      MARUTI.NS    0.068245
11         M&M.NS    0.092671

Top 3 safest stocks in FMCG:
            Stock  Risk_Score
15  HINDUNILVR.NS    0.015928
16         ITC.NS    0.039993
14       DABUR.NS    0.085644


In [41]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','BiLSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [ -0.17417684   3.89798089  73.03293813 -65.35838445]
Intercept: -0.05383513305556601


In [42]:
features_df['Risk_Score_GRU_Bi_Beta_GARCH'] = X @ weights

In [43]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (beta + GARCH + GRU+ bilstm):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GRU_Bi_Beta_GARCH')
          .head(3)[['Stock', 'Risk_Score_GRU_Bi_Beta_GARCH']])



Safest stocks in IT (beta + GARCH + GRU+ bilstm):
        Stock  Risk_Score_GRU_Bi_Beta_GARCH
3    WIPRO.NS                     -0.051728
1     INFY.NS                     -0.008001
0  HCLTECH.NS                     -0.007555

Safest stocks in Banking (beta + GARCH + GRU+ bilstm):
          Stock  Risk_Score_GRU_Bi_Beta_GARCH
7  KOTAKBANK.NS                     -0.026469
6  ICICIBANK.NS                     -0.025081
5   HDFCBANK.NS                      0.011042

Safest stocks in Pharma (beta + GARCH + GRU+ bilstm):
           Stock  Risk_Score_GRU_Bi_Beta_GARCH
9     DRREDDY.NS                     -0.001317
10  SUNPHARMA.NS                      0.000610
8       CIPLA.NS                      0.101521

Safest stocks in Auto (beta + GARCH + GRU+ bilstm):
            Stock  Risk_Score_GRU_Bi_Beta_GARCH
13  TATAMOTORS.NS                     -0.057265
12      MARUTI.NS                      0.064498
11         M&M.NS                      0.110726

Safest stocks in FMCG (beta + GARCH + GRU+ b

In [44]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'LSTM_Vol','BiLSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)

Learned Weights: [-0.18880999  1.32943712 28.86012729 26.67310512]
Intercept: -0.009608296278079687


In [45]:
features_df['Risk_Score_lstm_bi_garch_Beta'] = X @ weights

In [46]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (LSTM+ BiLSTM + GARCH + Beta ):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_lstm_bi_garch_Beta')
          .head(3)[['Stock', 'Risk_Score_lstm_bi_garch_Beta']])



Safest stocks in IT (LSTM+ BiLSTM + GARCH + Beta ):
        Stock  Risk_Score_lstm_bi_garch_Beta
3    WIPRO.NS                      -0.062882
2      TCS.NS                      -0.062786
0  HCLTECH.NS                      -0.047003

Safest stocks in Banking (LSTM+ BiLSTM + GARCH + Beta ):
          Stock  Risk_Score_lstm_bi_garch_Beta
7  KOTAKBANK.NS                      -0.074521
6  ICICIBANK.NS                      -0.043690
5   HDFCBANK.NS                      -0.018288

Safest stocks in Pharma (LSTM+ BiLSTM + GARCH + Beta ):
           Stock  Risk_Score_lstm_bi_garch_Beta
10  SUNPHARMA.NS                      -0.015591
9     DRREDDY.NS                       0.020242
8       CIPLA.NS                       0.042863

Safest stocks in Auto (LSTM+ BiLSTM + GARCH + Beta ):
            Stock  Risk_Score_lstm_bi_garch_Beta
13  TATAMOTORS.NS                      -0.098637
11         M&M.NS                      -0.014463
12      MARUTI.NS                       0.009726

Safest stocks in FMC

In [47]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)

Learned Weights: [ -0.21262152   3.76720293  52.8923556  -13.14196368]
Intercept: -0.051879274070611416


In [48]:
features_df['Risk_Score_GRU_LSTM_GARCH_Beta'] = X @ weights

In [49]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + Beta + GRU + LSTM):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GRU_LSTM_GARCH_Beta')
          .head(3)[['Stock', 'Risk_Score_GRU_LSTM_GARCH_Beta']])


Safest stocks in IT (GARCH + Beta + GRU + LSTM):
        Stock  Risk_Score_GRU_LSTM_GARCH_Beta
3    WIPRO.NS                       -0.039840
0  HCLTECH.NS                       -0.017117
1     INFY.NS                       -0.013613

Safest stocks in Banking (GARCH + Beta + GRU + LSTM):
          Stock  Risk_Score_GRU_LSTM_GARCH_Beta
7  KOTAKBANK.NS                       -0.034547
6  ICICIBANK.NS                       -0.019516
4   AXISBANK.NS                        0.034154

Safest stocks in Pharma (GARCH + Beta + GRU + LSTM):
           Stock  Risk_Score_GRU_LSTM_GARCH_Beta
10  SUNPHARMA.NS                       -0.002189
9     DRREDDY.NS                        0.015006
8       CIPLA.NS                        0.095128

Safest stocks in Auto (GARCH + Beta + GRU + LSTM):
            Stock  Risk_Score_GRU_LSTM_GARCH_Beta
13  TATAMOTORS.NS                       -0.068927
12      MARUTI.NS                        0.068245
11         M&M.NS                        0.092671

Safest stocks in

In [50]:
features_df[['Stock', 'Risk_Score', 'Risk_Score_lstm_bi_garch_Beta', 'Risk_Score_GRU_Bi_Beta_GARCH', 'Risk_Score_GRU_LSTM_GARCH_Beta']].head()

,Stock,Risk_Score,Risk_Score_lstm_bi_garch_Beta,Risk_Score_GRU_Bi_Beta_GARCH,Risk_Score_GRU_LSTM_GARCH_Beta
0,HCLTECH.NS,-0.017117,-0.047003,-0.007555,-0.017117
1,INFY.NS,-0.013613,-0.039926,-0.008001,-0.013613
2,TCS.NS,0.014233,-0.062786,0.038439,0.014233
3,WIPRO.NS,-0.039840,-0.062882,-0.051728,-0.039840
4,AXISBANK.NS,0.034154,0.009961,0.050434,0.034154


In [51]:

def compare_sector_models(sector_name, top_n=5):
    df = features_df[features_df['Sector'] == sector_name].copy()

    models = ['Risk_Score', 'Risk_Score_lstm_bi_garch_Beta', 'Risk_Score_GRU_Bi_Beta_GARCH', 'Risk_Score_GRU_LSTM_GARCH_Beta']

    print(f"\n📊 Sector: {sector_name}")
    for model in models:
        print(f"\n🔹 Top {top_n} safest stocks using {model}:")
        display(df.sort_values(model).head(top_n)[['Stock', model]])


In [52]:
compare_sector_models("IT", top_n=3)
compare_sector_models("Banking", top_n=3)
compare_sector_models("Pharma", top_n=3)



📊 Sector: IT

🔹 Top 3 safest stocks using Risk_Score:


,Stock,Risk_Score
3,WIPRO.NS,-0.039840
0,HCLTECH.NS,-0.017117
1,INFY.NS,-0.013613



🔹 Top 3 safest stocks using Risk_Score_lstm_bi_garch_Beta:


,Stock,Risk_Score_lstm_bi_garch_Beta
3,WIPRO.NS,-0.062882
2,TCS.NS,-0.062786
0,HCLTECH.NS,-0.047003



🔹 Top 3 safest stocks using Risk_Score_GRU_Bi_Beta_GARCH:


...

,Stock,Risk_Score_lstm_bi_garch_Beta
10,SUNPHARMA.NS,-0.015591
9,DRREDDY.NS,0.020242
8,CIPLA.NS,0.042863



🔹 Top 3 safest stocks using Risk_Score_GRU_Bi_Beta_GARCH:


,Stock,Risk_Score_GRU_Bi_Beta_GARCH
9,DRREDDY.NS,-0.001317
10,SUNPHARMA.NS,0.000610
8,CIPLA.NS,0.101521



🔹 Top 3 safest stocks using Risk_Score_GRU_LSTM_GARCH_Beta:


,Stock,Risk_Score_GRU_LSTM_GARCH_Beta
10,SUNPHARMA.NS,-0.002189
9,DRREDDY.NS,0.015006
8,CIPLA.NS,0.095128


In [53]:
def sector_ranking_table(sector_name):
    df = features_df[features_df['Sector'] == sector_name].copy()
    ranking_df = df[['Stock']].copy()

    for col in ['Risk_Score', 'Risk_Score_lstm_bi_garch_Beta', 'Risk_Score_GRU_Bi_Beta_GARCH', 'Risk_Score_GRU_LSTM_GARCH_Beta']:
        ranking_df[col + '_Rank'] = df[col].rank(method='min')

    return ranking_df.sort_values('Risk_Score_Rank')

In [54]:
sector_ranking_table("FMCG")

,Stock,Risk_Score_Rank,Risk_Score_lstm_bi_garch_Beta_Rank,Risk_Score_GRU_Bi_Beta_GARCH_Rank,Risk_Score_GRU_LSTM_GARCH_Beta_Rank
15,HINDUNILVR.NS,1.0,1.0,1.0,1.0
16,ITC.NS,2.0,2.0,2.0,2.0
14,DABUR.NS,3.0,3.0,3.0,3.0
